In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
#visualisation libraries
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use(style="seaborn")

import datetime
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import folium
from folium import plugins
from sklearn.model_selection import TimeSeriesSplit
import warnings
warnings.simplefilter("ignore")
from prophet import Prophet
from statsmodels.graphics.tsaplots import plot_acf

In [2]:
df_Germany= pd.read_csv("df_Germany.csv" ,encoding='unicode_escape')
df_Germany

,country,date,population,urbanPop%,#dailyTests,#testsTotal,#dailyTestsPerThousand,#testsTotalPerThousand,#tests7daySmooth,#tests7daySmoothPerThousand,...,E4InternationalSupport,H1PublicInformationCampaigns,H2TestingPolicy,H3ContactTracing,H4EmergencyInvestmentInHealthcare,H5InvestmentInVaccines,H6FacialCoverings,H7VaccinationPolicy,H8ProtectionOfElderlyPeople,mortality
0,Germany,2020-01-01,83783942,76%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,Germany,2020-01-02,83783942,76%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,Germany,2020-01-03,83783942,76%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,Germany,2020-01-04,83783942,76%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,Germany,2020-01-05,83783942,76%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1307,Germany,2021-09-24,83783942,76%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,3.0,2.0,0.0,0.0,2.0,5.0,2.0,0.022270
1308,Germany,2021-09-25,83783942,76%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,3.0,2.0,0.0,0.0,2.0,5.0,2.0,0.022243
1309,Germany,2021-09-26,83783942,76%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,3.0,2.0,0.0,0.0,2.0,5.0,2.0,0.022217
1310,Germany,2021-09-27,83783942,76%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.022216


In [4]:
#shift 21days mortality
df_Germany_mortality=df_Germany["mortality"]
df_Germany_mortality.drop([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21])

22      0.000000
23      0.000000
24      0.000000
25      0.000000
26      0.000000
          ...   
1307    0.022270
1308    0.022243
1309    0.022217
1310    0.022216
1311    0.000000
Name: mortality, Length: 1290, dtype: float64

In [5]:
#删除后21天
df_Germany.drop(df_Germany.index[1291:1312], inplace=True)
df_Germany

,country,date,population,urbanPop%,#dailyTests,#testsTotal,#dailyTestsPerThousand,#testsTotalPerThousand,#tests7daySmooth,#tests7daySmoothPerThousand,...,E4InternationalSupport,H1PublicInformationCampaigns,H2TestingPolicy,H3ContactTracing,H4EmergencyInvestmentInHealthcare,H5InvestmentInVaccines,H6FacialCoverings,H7VaccinationPolicy,H8ProtectionOfElderlyPeople,mortality
0,Germany,2020-01-01,83783942,76%,0.0,0.0,0.0,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,Germany,2020-01-02,83783942,76%,0.0,0.0,0.0,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,Germany,2020-01-03,83783942,76%,0.0,0.0,0.0,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,Germany,2020-01-04,83783942,76%,0.0,0.0,0.0,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,Germany,2020-01-05,83783942,76%,0.0,0.0,0.0,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1286,Germany,2021-09-03,83783942,76%,0.0,0.0,0.0,0.000,131739.0,1.570,...,0.0,2.0,3.0,2.0,0.0,0.0,2.0,5.0,2.0,0.023102
1287,Germany,2021-09-04,83783942,76%,0.0,0.0,0.0,0.000,133470.0,1.591,...,0.0,2.0,3.0,2.0,0.0,0.0,2.0,5.0,2.0,0.023003
1288,Germany,2021-09-05,83783942,76%,0.0,71313250.0,0.0,849.974,135200.0,1.611,...,0.0,2.0,3.0,2.0,0.0,0.0,2.0,5.0,2.0,0.023011
1289,Germany,2021-09-06,83783942,76%,0.0,0.0,0.0,0.000,136400.0,1.626,...,0.0,2.0,3.0,2.0,0.0,0.0,2.0,5.0,2.0,0.022986


In [6]:
df_Germany1=df_Germany.join(df_Germany_mortality,how="left",rsuffix="after21days")
df_Germany1

,country,date,population,urbanPop%,#dailyTests,#testsTotal,#dailyTestsPerThousand,#testsTotalPerThousand,#tests7daySmooth,#tests7daySmoothPerThousand,...,H1PublicInformationCampaigns,H2TestingPolicy,H3ContactTracing,H4EmergencyInvestmentInHealthcare,H5InvestmentInVaccines,H6FacialCoverings,H7VaccinationPolicy,H8ProtectionOfElderlyPeople,mortality,mortalityafter21days
0,Germany,2020-01-01,83783942,76%,0.0,0.0,0.0,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
1,Germany,2020-01-02,83783942,76%,0.0,0.0,0.0,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
2,Germany,2020-01-03,83783942,76%,0.0,0.0,0.0,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
3,Germany,2020-01-04,83783942,76%,0.0,0.0,0.0,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
4,Germany,2020-01-05,83783942,76%,0.0,0.0,0.0,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1286,Germany,2021-09-03,83783942,76%,0.0,0.0,0.0,0.000,131739.0,1.570,...,2.0,3.0,2.0,0.0,0.0,2.0,5.0,2.0,0.023102,0.023102
1287,Germany,2021-09-04,83783942,76%,0.0,0.0,0.0,0.000,133470.0,1.591,...,2.0,3.0,2.0,0.0,0.0,2.0,5.0,2.0,0.023003,0.023003
1288,Germany,2021-09-05,83783942,76%,0.0,71313250.0,0.0,849.974,135200.0,1.611,...,2.0,3.0,2.0,0.0,0.0,2.0,5.0,2.0,0.023011,0.023011
1289,Germany,2021-09-06,83783942,76%,0.0,0.0,0.0,0.000,136400.0,1.626,...,2.0,3.0,2.0,0.0,0.0,2.0,5.0,2.0,0.022986,0.022986


In [7]:
df_Germany1.to_csv("df_Germany_after21days",index=False)